In [ ]:
import urllib.request
import json
import re
import os
from pymystem3 import Mystem
import sqlite3
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from wordcloud import WordCloud


def create_db():

    conn = sqlite3.connect('lentach1.db')
    c = conn.cursor()
    c.execute(
        'CREATE TABLE IF NOT EXISTS Posts('
        'post_id, text, date, weekday, author_id)'
    )
    c.execute(
        'CREATE TABLE IF NOT EXISTS Comments('
        'post_id, comment_id, author_id, text)'
    )
    c.execute('CREATE TABLE IF NOT EXISTS Authors(author_id, city, age)')

    post_ids = []
    posts_text = ''
    comments = []
    token = 'ce9bbc81ce9bbc81ce9bbc81f7cef1aec5cce9bce9bbc8192236d' \
            '27163fe4bf62eb1946'
    req = urllib.request.Request(
        'https://api.vk.com/method/wall.get?owner_id=-29534144&count=100&'
        'access_token=%s&v=5.95' % token)
    req_2 = urllib.request.Request(
        'https://api.vk.com/method/wall.get?owner_id=-29534144&count=10&'
        'access_token=%s&offset=100&v=5.95' % token)
    response = urllib.request.urlopen(req)
    response_2 = urllib.request.urlopen(req_2)
    result = response.read().decode('utf-8')
    result_2 = response_2.read().decode('utf-8')
    data = json.loads(result)
    data_2 = json.loads(result_2)
    posts = data['response']['items']
    posts += data_2['response']['items']

    # Список с длинами постов
    len_posts = []

    #  Записывает информацию о постах в базу данных
    for post in posts:
        text = post['text']
        plain = re.sub('[^а-яА-Я\s]', '', text)
        len_post = len(plain.split())
        len_posts.append(len_post)
        posts_text += text + '\n'
        post_id = post['id']
        post_ids.append(post_id)
        date_vk = post['date']
        date = datetime.fromtimestamp(date_vk)
        author_id = post['from_id']
        weekday = date.weekday()

        c.execute('INSERT INTO Posts VALUES (?, ?, ?, ?, ?)',
                  (post_id, text, date, weekday, author_id))
        conn.commit()

    weekdays = ['понедельник', 'вторник', 'среда', 'четверг',
                'пятница', 'суббота', 'воскрсенье']
    len_weekday = []

    for i in range(0, 7):
        c.execute('SELECT text FROM Posts WHERE weekday='+str(i))
        text = c.fetchall()
        num = len(text)  # Количество постов в один день
        text = str(text)
        if len(text) == 2:
            plain = text
            av_weekday = 0
        else:
            plain = re.sub('[^а-яА-Я\s]', '', text)
            weekday = len(plain.split())  # Всего слов
            av_weekday = weekday // num
        len_weekday.append(av_weekday)

    #  График зависимости средней длины поста от дня недели
    #  Здесь в понедельник значение 0, потому что выкачанные 110 постов
    #  были опубликованы со вторника по воскресенье
    colors = ['red', 'orange', 'yellow', 'green', 'blue', 'violet', 'indigo']
    plt.bar(weekdays, len_weekday, color=colors)
    plt.title('График зависимости средней длины поста от дня недели')
    plt.ylabel('средняя длина поста')
    plt.xlabel('день недели')
    plt.xticks(weekdays, rotation=40, fontsize=6)
    plt.savefig('posts_weekday.png', fromat='png')
    plt.clf()

    c.execute('SELECT author_id FROM Posts')
    a = c.fetchall()
    post_au = []
    for a_i in a:
        if a_i not in post_au:
            post_au.append(a_i)

    c.execute('SELECT text FROM Posts WHERE author_id='+str(post_au)[2:-3])
    p = c.fetchall()
    n = len(p)  # количество постов от автора
    plain = re.sub('[^а-яА-Я\s]', '', str(p))
    au_p = len(plain.split())  # всего слов в постах одного автора
    author_p = au_p // n

    #  График зависимости средней длины поста от автора поста.
    #  Бесполезный, потому что все посты опубликованы
    #  от имени сообщества
    post_au = str(post_au)[2:-3]
    dot = '¯\_(o)_/¯'
    plt.bar(post_au, author_p, color='purple')
    plt.text(post_au, author_p, dot)
    plt.title('График зависимости средней длины поста от автора поста')
    plt.ylabel('средняя длина поста')
    plt.xlabel('автор сообщества')
    plt.savefig('posts_authors.png', fromat='png')
    plt.clf()

    directory = 'lentach'
    if not os.path.exists(directory):
        os.makedirs(directory)
    path = directory + '/posts.txt'
    with open(path, 'w+', encoding='utf-8') as f:
        f.write(posts_text)

    # Список со средней длинной комменатрия к посту
    av_com = []

    for post in post_ids:
        comment = []
        post_id = post
        reqc = urllib.request.Request(
            'https://api.vk.com/method/wall.getComments?post_id=%s&owner_id='
            '-29534144&count=100&access_token=%s&v=5.95' % (post_id, token))
        reqc_2 = urllib.request.Request(
            'https://api.vk.com/method/wall.getComments?post_id=%s&owner_id='
            '-29534144&count=10&access_token=%s&offset=100&v=5.95' %
            (post_id, token))
        response_c = urllib.request.urlopen(reqc)
        response_c2 = urllib.request.urlopen(reqc_2)
        result_c = response_c.read().decode('utf-8')
        result_c2 = response_c2.read().decode('utf-8')
        data_c = json.loads(result_c)
        data_c2 = json.loads(result_c2)
        comments += data_c['response']['items']
        comments += data_c2['response']['items']
        comment += data_c['response']['items'] + data_c2['response']['items']
        lengths = 0
        for com in comment:
            if 'deleted' in com:
                continue
            text = com['text']
            plain = re.sub('[^а-яА-Я\s]', '', text)
            len_com = len(plain.split())
            lengths += int(len_com)
        av_c = lengths // len(comment)
        av_com.append(av_c)

    # График зависимости длины поста от средней длины комменатрия
    length = dict(zip(len_posts, av_com))
    Y = [
        c[1] for c in sorted(
            length.items(), key=lambda x: x[0], reverse=False
            )
        ]
    X = sorted([l for l in length])
    plt.plot(X, Y, color='plum')
    plt.title('График зависимости длины поста от средней длины комменатрия')
    plt.ylabel('средняя длина комменатрия')
    plt.xlabel('длина поста')
    plt.savefig('posts_comments.png', fromat='png')
    plt.clf()

    com_text = ''
    authors = []

    # Записывает информацию о комментариях в базу данных
    for comment in comments:
        if 'deleted' in comment:
            continue
        text = comment['text']
        com_text += text + '\n'
        com_id = comment['id']
        post_id = comment['post_id']
        author_id = comment['from_id']
        authors.append(author_id)

        c.execute('INSERT INTO Comments VALUES (?, ?, ?, ?)',
                  (post_id, com_id, author_id, text))
        conn.commit()

    path = directory + '/comments.txt'
    with open(path, 'w+', encoding='utf-8') as f:
        f.write(com_text)

    #  Записывает информацию об авторах в базу данных
    for author in authors:
        if str(author).startswith('-'):
            author = str(author)[1:]
        user = 'id' + str(author)
        req = urllib.request.Request(
            'https://api.vk.com/method/users.get?v=5.95&access_token=%s&'
            'user_ids=%s&fields=city,bdate' % (token, user)
            )
        response = urllib.request.urlopen(req)
        result = response.read().decode('utf-8')
        data = json.loads(result)
        if 'bdate' in data['response'][0] and \
           len(data['response'][0]['bdate']) > 6:
            bdate = data['response'][0]['bdate']
            bdate = bdate.split('.')
            bdate = datetime(int(bdate[2]), int(bdate[1]), int(bdate[0]))
            current_time = datetime.today()
            age = (current_time - bdate).days // 365
        else:
            age = 'None'
        if 'city' in data['response'][0]:
            city = data['response'][0]['city']['title']
        else:
            city = 'None'

        c.execute(
            'INSERT INTO Authors VALUES (?, ?, ?)', (user, city, age)
            )
        conn.commit()

    comments_ages()
    comments_cities()
    top_words(posts_text)
    return posts_text


def top_words(posts_text):
    plain_texts = re.sub('[^а-яА-Я\s]', '', posts_text)
    plain_texts = plain_texts.lower()
    words = plain_texts.split()
    stop_words = ['в', 'и', 'на', 'с', 'об', 'к', 'но', 'что',
                  'другие', 'для', 'этому', 'это', 'которые', 'изза', 'как',
                  'все', 'или', 'кто', 'всех', 'которых', 'про', 'чтобы',
                  'там', 'может', 'его', 'они', 'если', 'уже', 'так', 'можно',
                  'также', 'том']
    words_without_stop = []
    for word in words:
        if word not in stop_words:
            words_without_stop.append(word)

    top = Counter(words_without_stop)
    top = dict(top)
    top = {c: top[c] for c in top if top[c] > 2 and len(c) > 2}
    top = (sorted(top.items(), key=lambda x: x[1], reverse=True))[:20]
    top = dict(top)
    top_nums = [c[1] for c in sorted(
        top.items(), key=lambda x: x[0], reverse=False
        )
        ]  # данные по оси y
    top_labs = sorted([t for t in top])  # подписи по оси x
    x = range(len(top_labs))

    plt.bar(x, top_nums, color='green')

    plt.ylabel('Количество употреблений\n', fontsize=10)
    plt.xticks(x, top_labs, rotation=60, fontsize=6)

    #  Сохраняет график топ-20 слов из постов
    plt.savefig('words.png', fromat='png')
    plt.clf()

    m = Mystem()
    lemmas = m.lemmatize(str(words_without_stop))

    stop_lem = ["', '", 'быть', 'который', 'сам', 'себя', 'она', 'такой',
                'этот', 'один', 'они', 'тот', 'свой', 'самый']
    lemmas_without_stop = []
    for lemma in lemmas:
        if lemma not in stop_lem:
            lemmas_without_stop.append(lemma)

    top_lem = Counter(lemmas_without_stop)
    top_lem = dict(top_lem)
    top_lem = {c: top_lem[c] for c in top_lem if top_lem[c] > 2 and len(c) > 2}
    top_lem = (sorted(top_lem.items(), key=lambda x: x[1], reverse=True))[:20]
    word_cloud = ''
    for i in dict(top_lem).keys():
        word_cloud += i + ' '
    top_lem = dict(top_lem)
    lem_nums = [c[1] for c in sorted(top_lem.items(),
                                     key=lambda x: x[0], reverse=False)]
    lem_labs = sorted([t for t in top_lem])
    y = range(len(lem_labs))

    plt.bar(y, lem_nums, color='lavender')
    plt.ylabel('Количество употреблений\n', fontsize=12)
    plt.xticks(y, lem_labs, rotation=60, fontsize=6)
    #  Сохраняет график топ-20 лемм из постов
    plt.savefig('lemmas.png', fromat='png')
    plt.clf()

    cloud = WordCloud(background_color='white')
    cloud.generate(word_cloud)
    plt.imshow(cloud, interpolation='bilinear')
    plt.axis("off")

    cloud.to_file('lemmas_cloud.png')


def comments_ages():
    age = []  # Сколько вообще вариантов возрастов авторов бывает
    age_comments = []

    conn = sqlite3.connect('lentach.db')
    c = conn.cursor()

    # Собираем информацию для графика про возраст + длина поста
    c.execute('SELECT age FROM Authors')
    ages = c.fetchall()
    # print(type(ages))
    for ag in ages:
        ag = str(ag)[1:-2]
        if ag not in age:
            age.append(ag)

    lengths = []  # Средняя длина комментария
    for a in age:
        len_com = 0  # Всего комментариев от людей одного возраста
        words_com = 0  # Всего слов в комментариях от людей одного возраста
        age_com = []  # Список из всех комметариев от людей одного возраста
        c.execute('SELECT author_id FROM Authors WHERE age=' + str(a))
        ids = c.fetchall()  # Все комментаторы этого возраста
        for i in ids:
            i = str(i)[4:-3]
            c.execute('SELECT text FROM Comments WHERE author_id=' + i)
            com = c.fetchall()  # Все комментарии этого автора
            plain = re.sub('[^а-яА-Я\s]', '', str(com))
            words_plain = len(plain.split())
            words_com += words_plain
            age_com.append(com)
        len_com = len(age_com)
        age_comments.append(age_com)
        aver_com = words_com // len_com
        lengths.append(aver_com)

    comment_age = dict(zip(age, lengths))
    Y = [c[1] for c in sorted(
        comment_age.items(), key=lambda x: x[0], reverse=False
        )
        ]
    X = sorted([l for l in comment_age])
    plt.plot(X, Y, color='aqua')
    plt.title('График зависимости длины комменатрия от возраста автора')
    plt.ylabel('средняя длина комменатрия')
    plt.xlabel('возраст автора')
    plt.xticks(np.arange(0, int(X[96]), 10))
    plt.savefig('comments_ages.png', fromat='png')
    plt.clf()


def comments_cities():
    city = []  # Сколько вообще вариантов городов авторов бывает
    city_comments = []

    conn = sqlite3.connect('lentach.db')
    c = conn.cursor()

    # Собираем информацию для графика про город + длина поста
    c.execute('SELECT city FROM Authors')
    cities = c.fetchall()
    for ci in cities:
        ci = str(ci)[1:-2]
        if ci not in city:
            city.append(ci)

    lengths = []  # Средняя длина комментария
    for ci in city:
        len_com = 0  # Всего комментариев от людей из одного города
        words_com = 0  # Всего слов в комментариях от людей из одного города
        city_com = []  # Список из всех комметариев от людей из одного города
        c.execute('SELECT author_id FROM Authors WHERE city='+str(ci))
        ids = c.fetchall()  # Все комментаторы этого возраста
        for i in ids:
            i = str(i)[4:-3]
            c.execute('SELECT text FROM Comments WHERE author_id=' + i)
            com = c.fetchall()  # Все комментарии этого автора
            plain = re.sub('[^а-яА-Я\s]', '', str(com))
            words_plain = len(plain.split())
            words_com += words_plain
            city_com.append(com)
        len_com = len(city_com)
        city_comments.append(city_com)
        aver_com = words_com // len_com
        lengths.append(aver_com)

    comment_city = dict(zip(city, lengths))
    city_names = comment_city.keys()
    Y = [c[1] for c in sorted(
        comment_city.items(), key=lambda x: x[0], reverse=False
        )
        ]
    X = sorted([l for l in comment_city])
    for x, y, d in zip(X, Y, city_names):
        plt.scatter(x, y)
        plt.text(x, y, d)
    plt.title('График зависимости длины комменатрия от города')
    plt.ylabel('средняя длина комменатрия')
    plt.xlabel('город')
    plt.savefig('comments_cities.png', fromat='png')
    plt.clf()

if __name__ == '__main__':
    create_db()
